In [4]:
# coding: utf-8
import os
import sys
import math
import torch
import numpy as np
import pandas as pd
from tensorboard.backend.event_processing import event_accumulator

In [1]:
task_name_list = ['Multi_Tasks_Samll','Multi_Tasks_Medium', 'Multi_Tasks_Big', 'Multi_Tasks_Large']#  
task_id_list = [1,4,6,8,11,19] + [0,2,10,15,20,23] + [12,13,17,18,22,24] + [3,5,7,9,14,16,21] # 
run_name = "_run_0"
task_num = 6
file_name_list = ["./result/stop_index/run-3C_GAFSE_" + task_name + run_name + "-tag-" for task_name in task_name_list]
print(len(file_name_list))
print(len(task_id_list))

4
25


In [2]:
def getFlist(path):
    for root, dirs, files in os.walk(file_dir):
        print('root_dir:', root) 
        print('sub_dirs:', dirs)   
        print('files:', files)   
    return files
    
def readEvent(event_path, scalarName):
    event = event_accumulator.EventAccumulator(event_path)
    event.Reload()
    value = event.scalars.Items(scalarName)
    return value

def exportToexcel(scalarNameList, excelName,task_name,run_name):
    for scalarName in scalarNameList:
        try:
            scalarValue = readEvent(event_path,scalarName)
            data = pd.DataFrame(scalarValue)
            saveFile = './result/stop_index/run-3C_GAFSE_'+task_name+run_name+'-tag-'+scalarName.replace("/", "_")+'.csv'
            data.to_csv(path_or_buf=saveFile,index=False)
            print("Scalar saved in:",saveFile)
        except Exception as e:
            print('Error:',e.__class__.__name__,e)
            continue

scalarNameList = []
# scalarNameList = [f'lr/AFSE_lr', f'lr/max_lr', f'lr/punish_lr']
for i in range(task_num):
    scalarNameList.extend([f'val/T{i}_auc', f'val/T{i}_acc', f'val/T{i}_mcc', f'val/T{i}_sp', f'val/T{i}_sn', 
                           f'test/T{i}_auc', f'test/T{i}_acc', f'test/T{i}_mcc', f'test/T{i}_sp', f'test/T{i}_sn'])
for task_name in task_name_list:
    event_path ="./log/3C_GAFSE_" + task_name + run_name
    excelName = "./result/stop_index/3C_GAFSE_" + task_name + run_name + ".csv"
    exportToexcel(scalarNameList,excelName,task_name,run_name)

In [4]:
task_name = 'Multi_Tasks_Large'
i = 6
scalarNameList = [f'val/T{i}_auc', f'val/T{i}_acc', f'val/T{i}_mcc', f'val/T{i}_sp', f'val/T{i}_sn', 
                           f'test/T{i}_auc', f'test/T{i}_acc', f'test/T{i}_mcc', f'test/T{i}_sp', f'test/T{i}_sn']
event_path ="./log/3C_GAFSE_" + task_name + run_name
excelName = "./result/stop_index/3C_GAFSE_" + task_name + run_name + ".csv"
exportToexcel(scalarNameList,excelName,task_name,run_name)

In [9]:
task_num = 6
all_task_num = len(task_id_list)
stop_epoch_list = [0 for i in range(all_task_num)]
stop_epoch_best = [0 for i in range(all_task_num)]
max_epoch_list = [0 for i in range(all_task_num)]
total_index_list = [0 for i in range(all_task_num)]
best_test_index = 0
best_epoch_list = [0 for i in range(all_task_num)]
test_auc_list = [0 for i in range(all_task_num)]
test_acc_list = [0 for i in range(all_task_num)]
test_mcc_list = [0 for i in range(all_task_num)]
test_sp_list = [0 for i in range(all_task_num)]
test_sn_list = [0 for i in range(all_task_num)]
test_auc_best = [0 for i in range(all_task_num)]
test_acc_best = [0 for i in range(all_task_num)]
test_mcc_best = [0 for i in range(all_task_num)]
test_sp_best = [0 for i in range(all_task_num)]
test_sn_best = [0 for i in range(all_task_num)]
for i, file_name in enumerate(file_name_list):
    task_num_range = task_num
    if i == len(file_name_list)-1:
        task_num_range = task_num + 1
    for k in range(task_num_range):
        val_auc, val_acc, val_mcc, val_sp, val_sn = [0,0,0,0,0]
        val_auc_df = pd.read_csv(file_name+f"val_T{k}_auc.csv", header=0, names = ["Step","Value"],usecols=[1,2])
        step_list = np.array(val_auc_df.Step)
        epoch = len(step_list)
        val_auc = np.array(val_auc_df.Value[:epoch])
        val_acc_df = pd.read_csv(file_name+f"val_T{k}_acc.csv", header=0, names = ["Value"],usecols=[2])
        val_acc = np.array(val_acc_df.Value[:epoch])
        val_mcc_df = pd.read_csv(file_name+f"val_T{k}_mcc.csv", header=0, names = ["Value"],usecols=[2])
        val_mcc = np.array(val_mcc_df.Value[:epoch])
        val_sp_df = pd.read_csv(file_name+f"val_T{k}_sp.csv", header=0, names = ["Value"],usecols=[2])
        val_sp = np.array(val_sp_df.Value[:epoch])
        val_sn_df = pd.read_csv(file_name+f"val_T{k}_sn.csv", header=0, names = ["Value"],usecols=[2])
        val_sn = np.array(val_sn_df.Value[:epoch])
        test_auc_df = pd.read_csv(file_name+f"test_T{k}_auc.csv", header=0, names = ["Value"],usecols=[2])
        test_auc_task = np.array(test_auc_df.Value[:epoch])
        test_acc_df = pd.read_csv(file_name+f"test_T{k}_acc.csv", header=0, names = ["Value"],usecols=[2])
        test_acc_task = np.array(test_acc_df.Value[:epoch])
        test_mcc_df = pd.read_csv(file_name+f"test_T{k}_mcc.csv", header=0, names = ["Value"],usecols=[2])
        test_mcc_task = np.array(test_mcc_df.Value[:epoch])
        test_sp_df = pd.read_csv(file_name+f"test_T{k}_sp.csv", header=0, names = ["Value"],usecols=[2])
        test_sp_task = np.array(test_sp_df.Value[:epoch])
        test_sn_df = pd.read_csv(file_name+f"test_T{k}_sn.csv", header=0, names = ["Value"],usecols=[2])
        test_sn_task = np.array(test_sn_df.Value[:epoch])

        counter = 0
        patience = 13
#                         print(val_auc)
        stop_index = [(val_auc[i] + val_acc[i] + val_sp[i] + val_sn[i]) for i in range(epoch)] #  + b*val_acc[i] + c*val_mcc[i] + d*val_sp[i] + e*val_sn[i]
#                             print(stop_index)
        val_best_index = 0
        for j in range(epoch-1):
            if math.isnan(stop_index[j+1]):
                stop_index[j+1] = 0
            if stop_index[j+1] >= val_best_index:
                val_best_index = stop_index[j+1]
                counter = 0
                val_best_epoch = j + 1
                stop_epoch_list[i*task_num+k] = val_best_epoch
                max_epoch_list[i*task_num+k] = epoch
            else:
                counter += 1
            if counter >= patience:
                break

        total_index_list[i*task_num+k] = test_auc_task[val_best_epoch] + test_acc_task[val_best_epoch] + test_mcc_task[val_best_epoch] + test_sp_task[val_best_epoch] + test_sn_task[val_best_epoch]
        test_auc_list[i*task_num+k] = test_auc_task[val_best_epoch]
        test_acc_list[i*task_num+k] = test_acc_task[val_best_epoch]
        test_mcc_list[i*task_num+k] = test_mcc_task[val_best_epoch]
        test_sp_list[i*task_num+k] = test_sp_task[val_best_epoch]
        test_sn_list[i*task_num+k] = test_sn_task[val_best_epoch]

test_auc_average = np.array(test_auc_list).mean()
test_acc_average = np.array(test_acc_list).mean()
test_mcc_average = np.array(test_mcc_list).mean()
test_sp_average = np.array(test_sp_list).mean()
test_sn_average = np.array(test_sn_list).mean()
print("Average AUC: %.3f"%test_auc_average, "ACC: %.3f"%test_acc_average, "MCC: %.3f"%test_mcc_average, 
                          "Sp: %.3f"%test_sp_average, "Sn: %.3f"%test_sn_average)

Performace_df = pd.DataFrame([])     
Performace_df["Task ID"] = np.array(task_id_list)
Performace_df["Stop Epoch"] = np.array(stop_epoch_list)+1
Performace_df["Total Epoch"] = np.array(max_epoch_list)
Performace_df["auc"] = np.array(test_auc_list)
Performace_df["acc"] = np.array(test_acc_list)
Performace_df["mcc"] = np.array(test_mcc_list)
Performace_df["sp"] = np.array(test_sp_list)
Performace_df["sn"] = np.array(test_sn_list)
Performace_df.to_csv(path_or_buf='./result/GAFSE-MP.csv')
print("Performance of GAFSE-MP successfully saved!")

Average AUC: 0.880 ACC: 0.855 MCC: 0.553 Sp: 0.844 Sn: 0.762
Performance of GAFSE-MP successfully saved!
